# 🔄 Model Loading Tutorial

This tutorial demonstrates how to **load semantic data from RDF graphs** into Python objects using the `ModelLoader` class.

## What You'll Learn

1. **Creating sample RDF data** for testing
2. **Loading instances** from RDF graphs
3. **Working with query results** as DataFrames
4. **Loading multiple entity types** at once
5. **Handling complex properties** with units and values
6. **Custom SPARQL queries** and manual querying

This is the reverse of template generation - instead of creating RDF from Python objects, we're loading Python objects from existing RDF data! 🔄

## Setup and Imports

In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
from semantic_objects.model_loader import ModelLoader, query_to_df
from semantic_objects.s223.entities import Space, Window
from semantic_objects.s223.properties import Area, Azimuth, Tilt
from semantic_objects.namespaces import S223, QUDT, UNIT, bind_prefixes
import pandas as pd
from pprint import pprint

print("✅ Imports successful!")

## 1. 🏗️ Creating Sample RDF Data

First, let's create some sample RDF data to work with. In a real scenario, this would be loaded from a file or database.

In [ ]:
def create_sample_graph():
    """Create a sample RDF graph with spaces and windows for testing."""
    g = Graph()
    bind_prefixes(g)
    
    # Define a namespace for our test data
    EX = Namespace("http://example.org/building#")
    g.bind("ex", EX)
    
    # Create a Space with an Area property
    space1 = EX["Space1"]
    g.add((space1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Space"]))
    
    # Create Area property for the space
    area1 = EX["Space1_Area"]
    g.add((area1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((area1, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((area1, S223["hasValue"], Literal(100.0)))
    g.add((area1, QUDT["hasUnit"], UNIT["FT2"]))
    g.add((space1, S223["hasProperty"], area1))
    
    # Create another Space
    space2 = EX["Space2"]
    g.add((space2, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Space"]))
    
    area2 = EX["Space2_Area"]
    g.add((area2, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((area2, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((area2, S223["hasValue"], Literal(150.0)))
    g.add((area2, QUDT["hasUnit"], UNIT["M2"]))
    g.add((space2, S223["hasProperty"], area2))
    
    # Create a Window with multiple properties
    window1 = EX["Window1"]
    g.add((window1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Window"]))
    
    # Window area
    window_area = EX["Window1_Area"]
    g.add((window_area, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_area, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((window_area, S223["hasValue"], Literal(10.0)))
    g.add((window_area, QUDT["hasUnit"], UNIT["FT2"]))
    g.add((window1, S223["hasProperty"], window_area))
    
    # Window azimuth
    window_azimuth = EX["Window1_Azimuth"]
    g.add((window_azimuth, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_azimuth, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Azimuth")))
    g.add((window_azimuth, S223["hasValue"], Literal(180.0)))
    g.add((window_azimuth, QUDT["hasUnit"], UNIT["DEG"]))
    g.add((window1, S223["hasProperty"], window_azimuth))
    
    # Window tilt
    window_tilt = EX["Window1_Tilt"]
    g.add((window_tilt, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_tilt, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Tilt")))
    g.add((window_tilt, S223["hasValue"], Literal(90.0)))
    g.add((window_tilt, QUDT["hasUnit"], UNIT["DEG"]))
    g.add((window1, S223["hasProperty"], window_tilt))
    
    return g

# Create the sample graph
sample_graph = create_sample_graph()
print(f"✅ Created sample graph with {len(sample_graph)} triples")

In [ ]:
# Let's look at the RDF data we created
print("Sample RDF data (Turtle format):")
print(sample_graph.serialize(format='turtle'))

## 2. 🔍 SPARQL Query Generation

Before loading instances, let's see how the library generates SPARQL queries from Resource classes:

In [ ]:
# Generate query for Space class
space_query = Space.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Space class:")
print(space_query)
print("\n" + "="*80)

In [ ]:
# Generate query for Window class (more complex)
window_query = Window.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Window class:")
print(window_query)

## 3. 📊 Querying for Data

Now let's use the `ModelLoader` to query our sample graph:

In [ ]:
# Initialize ModelLoader with our sample graph
loader = ModelLoader(source=sample_graph)
print("✅ ModelLoader initialized")

In [ ]:
# Query for Space instances (returns DataFrame)
print("Querying for Space instances...")
space_df = loader.query_class(Space, ontology='s223')
print(f"Found {len(space_df)} Space instances")

if not space_df.empty:
    print("\nDataFrame columns:", space_df.columns.tolist())
    print("\nSpace query results:")
    print(space_df)
else:
    print("No spaces found")

In [ ]:
# Query for Window instances
print("Querying for Window instances...")
window_df = loader.query_class(Window, ontology='s223')
print(f"Found {len(window_df)} Window instances")

if not window_df.empty:
    print("\nWindow query results:")
    print(window_df)
else:
    print("No windows found")

## 4. 🏗️ Loading Python Objects

Now let's load the query results as actual Python objects:

In [ ]:
# Load Space instances as Python objects
print("Loading Space instances as Python objects...")
spaces = loader.load_instances(Space, ontology='s223')
print(f"Loaded {len(spaces)} Space objects")

for i, space in enumerate(spaces):
    print(f"\nSpace {i+1}:")
    print(f"  Name: {space._name}")
    print(f"  Type: {type(space).__name__}")
    if hasattr(space, 'area') and space.area:
        print(f"  Area: {space.area.value} {space.area.unit}")
        print(f"  Area type: {type(space.area).__name__}")

In [ ]:
# Load Window instances as Python objects
print("Loading Window instances as Python objects...")
windows = loader.load_instances(Window, ontology='s223')
print(f"Loaded {len(windows)} Window objects")

for i, window in enumerate(windows):
    print(f"\nWindow {i+1}:")
    print(f"  Name: {window._name}")
    print(f"  Type: {type(window).__name__}")
    
    if hasattr(window, 'area') and window.area:
        print(f"  Area: {window.area.value} {window.area.unit}")
    if hasattr(window, 'azimuth') and window.azimuth:
        direction = {
            0.0: "North", 90.0: "East", 180.0: "South", 270.0: "West"
        }.get(window.azimuth.value, f"{window.azimuth.value}°")
        print(f"  Azimuth: {window.azimuth.value}° ({direction})")
    if hasattr(window, 'tilt') and window.tilt:
        orientation = "Vertical" if window.tilt.value == 90.0 else f"{window.tilt.value}° tilt"
        print(f"  Tilt: {window.tilt.value}° ({orientation})")

## 5. 🔄 Loading Multiple Classes

You can load multiple entity types in a single operation:

In [ ]:
# Load multiple classes at once
print("Loading multiple classes...")
results = loader.load_multiple_classes(
    {
        'spaces': Space,
        'windows': Window
    },
    ontology='s223'
)

print(f"\nResults:")
for key, instances in results.items():
    print(f"  {key}: {len(instances)} instances")
    for instance in instances:
        print(f"    - {instance._name} ({type(instance).__name__})")

## 6. 🔧 Working with Properties

Let's examine the properties of our loaded objects in detail:

In [ ]:
# Examine a space's area property in detail
if spaces:
    space = spaces[0]
    print(f"Examining space: {space._name}")
    print(f"Space area object: {space.area}")
    print(f"Area value: {space.area.value}")
    print(f"Area unit: {space.area.unit}")
    print(f"Area quantity kind: {space.area.qk}")
    print(f"Unit IRI: {space.area.unit._get_iri()}")
    print(f"Quantity kind IRI: {space.area.qk._get_iri()}")

In [ ]:
# Examine a window's properties in detail
if windows:
    window = windows[0]
    print(f"Examining window: {window._name}")
    
    properties = ['area', 'azimuth', 'tilt']
    for prop_name in properties:
        if hasattr(window, prop_name):
            prop = getattr(window, prop_name)
            if prop:
                print(f"\n{prop_name.title()}:")
                print(f"  Value: {prop.value}")
                print(f"  Unit: {prop.unit}")
                print(f"  Quantity Kind: {prop.qk}")
                print(f"  Type: {type(prop).__name__}")

## 7. 🔍 Manual SPARQL Queries

You can also execute custom SPARQL queries directly:

In [ ]:
# Custom SPARQL query to find all properties
custom_query = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?entity ?property ?value ?unit ?qk WHERE {
    ?entity s223:hasProperty ?property .
    ?property s223:hasValue ?value .
    ?property qudt:hasUnit ?unit .
    ?property qudt:hasQuantityKind ?qk .
}
"""

# Execute the query
properties_df = query_to_df(custom_query, sample_graph)
print("All properties in the graph:")
print(properties_df)

In [ ]:
# Query to find entities by type
entity_query = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?entity ?type WHERE {
    ?entity rdf:type ?type .
    FILTER(?type IN (s223:Space, s223:Window))
}
"""

entities_df = query_to_df(entity_query, sample_graph)
print("\nAll entities by type:")
print(entities_df)

## 8. 📁 Loading from Files

In practice, you'll often load RDF data from files. Let's save our sample data and load it back:

In [ ]:
# Save sample graph to file
sample_graph.serialize(destination='sample_building.ttl', format='turtle')
print("✅ Saved sample data to 'sample_building.ttl'")

# Load from file
file_loader = ModelLoader(source='sample_building.ttl')
print("✅ Loaded ModelLoader from file")

# Load spaces from file
file_spaces = file_loader.load_instances(Space, ontology='s223')
print(f"Loaded {len(file_spaces)} spaces from file")

for space in file_spaces:
    print(f"  - {space._name}: {space.area.value} {space.area.unit}")

## 9. 🔄 Round-Trip: Generate and Load

Let's demonstrate a complete round-trip: create objects, generate RDF, then load them back:

In [ ]:
# Step 1: Create objects using BMotifSession
from semantic_objects.build_model import BMotifSession

session = BMotifSession(ns='roundtrip')
session.load_class_templates(Space)
session.load_class_templates(Window)

# Create some objects
office = Space(area=120.0)
office._name = "Office_RT1"

east_window = Window(area=15.0, azimuth=90.0, tilt=90.0)
east_window._name = "Window_RT1"

# Evaluate them (generate RDF)
session.evaluate(office)
session.evaluate(east_window)

print("✅ Created and evaluated objects")
print(f"Generated graph has {len(session.graph)} triples")

In [ ]:
# Step 2: Load the objects back from the generated RDF
roundtrip_loader = ModelLoader(source=session.graph)

# Load spaces and windows
loaded_spaces = roundtrip_loader.load_instances(Space, ontology='s223')
loaded_windows = roundtrip_loader.load_instances(Window, ontology='s223')

print(f"\nLoaded back {len(loaded_spaces)} spaces and {len(loaded_windows)} windows")

# Compare original and loaded objects
print("\nOriginal vs Loaded:")
print(f"Original office area: {office.area.value} {office.area.unit}")
if loaded_spaces:
    print(f"Loaded office area: {loaded_spaces[0].area.value} {loaded_spaces[0].area.unit}")

print(f"\nOriginal window azimuth: {east_window.azimuth.value}°")
if loaded_windows:
    print(f"Loaded window azimuth: {loaded_windows[0].azimuth.value}°")

## 10. 🎯 Advanced Usage Tips

Here are some advanced patterns for working with the ModelLoader:

In [ ]:
# Tip 1: Filter loaded objects
large_spaces = [space for space in spaces if space.area.value > 120.0]
print(f"Found {len(large_spaces)} large spaces (>120 ft²)")

# Tip 2: Group by property values
from collections import defaultdict
spaces_by_unit = defaultdict(list)
for space in spaces:
    unit = str(space.area.unit)
    spaces_by_unit[unit].append(space)

print("\nSpaces grouped by unit:")
for unit, space_list in spaces_by_unit.items():
    print(f"  {unit}: {len(space_list)} spaces")

In [ ]:
# Tip 3: Convert to pandas for analysis
if spaces:
    space_data = []
    for space in spaces:
        space_data.append({
            'name': space._name,
            'area_value': space.area.value,
            'area_unit': str(space.area.unit)
        })
    
    spaces_analysis_df = pd.DataFrame(space_data)
    print("Spaces analysis DataFrame:")
    print(spaces_analysis_df)
    
    # Basic statistics
    print(f"\nTotal area: {spaces_analysis_df['area_value'].sum()}")
    print(f"Average area: {spaces_analysis_df['area_value'].mean():.1f}")
    print(f"Max area: {spaces_analysis_df['area_value'].max()}")

In [ ]:
# Tip 4: Error handling for missing properties
def safe_get_property(obj, prop_name, default="N/A"):
    """Safely get a property value with fallback"""
    if hasattr(obj, prop_name):
        prop = getattr(obj, prop_name)
        if prop and hasattr(prop, 'value'):
            return f"{prop.value} {prop.unit}"
    return default

# Use with windows (which might not have all properties)
print("Window properties (safe access):")
for window in windows:
    print(f"  {window._name}:")
    print(f"    Area: {safe_get_property(window, 'area')}")
    print(f"    Azimuth: {safe_get_property(window, 'azimuth')}")
    print(f"    Tilt: {safe_get_property(window, 'tilt')}")

## 📊 Summary

Congratulations! You've learned how to use the ModelLoader to work with semantic data:

✅ **Created sample RDF data** with spaces and windows  
✅ **Generated SPARQL queries** automatically from class definitions  
✅ **Queried RDF graphs** and got results as DataFrames  
✅ **Loaded Python objects** from RDF data with full type safety  
✅ **Loaded multiple entity types** in single operations  
✅ **Worked with properties** including values, units, and quantity kinds  
✅ **Executed custom SPARQL queries** for advanced use cases  
✅ **Loaded from files** and performed round-trip operations  
✅ **Applied advanced patterns** for filtering and analysis  

### Key Benefits of ModelLoader

1. **Type Safety**: Loaded objects have full type hints and validation
2. **Automatic Queries**: No need to write SPARQL manually
3. **Property Handling**: Complex properties with units are handled automatically
4. **Flexible Sources**: Works with files, graphs, or databases
5. **Integration**: Seamless integration with BuildingMOTIF and other tools

### Next Steps

- **Template Generation Tutorial**: Learn to create BuildingMOTIF templates
- **Advanced Examples**: Complex relationships and custom queries
- **Custom Entities**: Create your own semantic object types
- **Validation**: Use SHACL shapes for data validation

The ModelLoader bridges the gap between semantic RDF data and Pythonic object-oriented programming! 🌉